In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv("/Users/alexis/Cranfield/AI/assignment/data/train.csv")
val_df = pd.read_csv("/Users/alexis/Cranfield/AI/assignment/data/val.csv")
test_df = pd.read_csv("/Users/alexis/Cranfield/AI/assignment/data/test.csv")

In [ ]:
train_df.head()

In [ ]:
val_df.head()

In [ ]:
test_df.head()

In [ ]:
print("Training size:", train_df.shape)
print("Validation size :", val_df.shape)
print("Test size: ", test_df.shape)

In [ ]:
import numpy as np

print("Training labels:", np.bincount(train_df['class_id']))
print("Validation labels:", np.bincount(val_df['class_id']))
print("Test labels:", np.bincount(test_df['class_id']))

### Labels Distribution

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

classes = train_df['class_id'].unique()
classes.sort()
class_counts = train_df['class_id'].value_counts().sort_index()
plt.figure(figsize=(10, 5))
sns.barplot(x=class_counts.index, y=class_counts.values)
plt.title('Class distribution in the training set', fontsize=15)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Class ID', fontsize=12)
plt.show()

In [ ]:
print("Training set class distribution")
train_df['class_id'].value_counts().sort_index()

### Zero-rule baseline (majority class classifier)

In [ ]:
from collections import Counter

train_counter = Counter()
for class_id in train_df['class_id']:
    train_counter.update([class_id])

val_counter = Counter()
for class_id in val_df['class_id']:
    val_counter.update([class_id])

test_counter = Counter()
for class_id in test_df['class_id']:
    test_counter.update([class_id])

print("\nTraining label distribution:")
print(sorted(train_counter.items()))

print("\nValidation label distribution:")
print(sorted(val_counter.items()))

print("\nTest label distribution:")
print(sorted(test_counter.items()))

In [ ]:
majority_class = test_counter.most_common(1)[0]
print("Majority class:", majority_class[0])

baseline_acc = majority_class[1] / sum(test_counter.values())
print("Accuracy when always predicting the majority class:")
print(f"{baseline_acc:.2f} ({baseline_acc*100:.2f}%)")

In [1]:
import torch

torch.manual_seed(123)

In [2]:
from ChestXrayDataModule import ChestXrayDataModule

train_df_path = "/Users/alexis/Cranfield/AI/assignment/data/train.csv"
val_df_path = "/Users/alexis/Cranfield/AI/assignment/data/val.csv"
test_df_path = "/Users/alexis/Cranfield/AI/assignment/data/test.csv"
hdf5_path = "/Volumes/ALEXIS/ai_project_cranfield/dicom_images_final.hdf5"

dataModule = ChestXrayDataModule(
    train_dataset_path=train_df_path,
    val_dataset_path=val_df_path,
    test_dataset_path=test_df_path,
    hdf5_path=hdf5_path,
    batch_size=4,
    num_workers=8
)

In [3]:
from ChestXrayLightningModel import ChestXrayLightningModel

model = ChestXrayLightningModel(
    learning_rate=0.005,
    num_classes=15
)

In [4]:
import lightning as L

trainer = L.Trainer(
    max_epochs=10,
    accelerator="gpu",
    devices="auto",
    deterministic=True,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/alexis/Cranfield/AI/assignment/chest-x-ray-abnormalities-detection/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [ ]:
""" model.train()
dataModule.setup("fit")
trainer.fit(model, datamodule=dataModule) """

In [5]:
dataModule.setup("test")

In [6]:
import numpy as np


def format_prediction_string(labels, boxes, scores):
    pred_strings = []
    for j in zip(labels, scores, boxes):
        pred_strings.append(
            "{0} {1:.4f} {2} {3} {4} {5}".format(
                j[0], j[1], j[2][0], j[2][1], j[2][2], j[2][3]
            )
        )
    return " ".join(pred_strings)

In [7]:
model = ChestXrayLightningModel.load_from_checkpoint(
    "/Users/alexis/Cranfield/AI/assignment/Faster-R-CNN/model/version_14/checkpoints/epoch=9-step=9260.ckpt",
    num_classes=15,
)

In [8]:
detection_threshold = 0.4
results = []
model.model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [9]:
# Ensure the model is on GPU if available
model.to('cuda' if torch.cuda.is_available() else 'cpu')

ChestXrayLightningModel(
  (model): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=0.0)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=0.0)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=0.0)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  

In [11]:
from tqdm import tqdm

results = []
with torch.inference_mode():
    for image_ids, images, _ in tqdm(dataModule.test_dataloader(), desc="Evaluating"):
        outputs = model(images)
        for i, output in enumerate(outputs):
            image_id = image_ids[i]
            result = {"image_id": image_id, "PredictionString": "14 1.0 0 0 1 1"}

            boxes = output["boxes"].data.cpu().numpy()
            labels = output["labels"].data.cpu().numpy() - 1
            scores = output["scores"].data.cpu().numpy()

            # Assuming detection_threshold is defined
            valid = scores >= detection_threshold

            if valid.any():
                result = {
                    "image_id": image_id,
                    "PredictionString": format_prediction_string(
                        labels[valid], boxes[valid], scores[valid]
                    ),
                }
            
            print(result)

            results.append(result)


df = pd.DataFrame(results, columns=["image_id", "PredictionString"])

Evaluating:   0%|          | 1/1757 [00:18<8:55:26, 18.30s/it]

{'image_id': '27b822c5d3b354f096dfb788fd3fa636', 'PredictionString': '8 0.8928 69.38096618652344 116.62224578857422 86.6138687133789 133.91175842285156 0 0.8732 112.89633178710938 70.56173706054688 138.8718719482422 100.48268127441406 8 0.8073 175.08900451660156 145.5911102294922 181.0615997314453 151.89627075195312 8 0.7882 35.42214584350586 124.0315933227539 40.95875930786133 131.5504913330078 8 0.7735 168.79669189453125 109.6122055053711 175.3958740234375 114.38080596923828 8 0.7421 51.692481994628906 116.00086212158203 58.26260757446289 120.69606018066406 3 0.7322 83.06791687011719 137.2764434814453 163.2445831298828 168.31044006347656 8 0.6729 34.32096862792969 162.9665069580078 41.514930725097656 169.59893798828125 8 0.6681 46.89222717285156 105.73015594482422 52.8055534362793 110.87127685546875 8 0.5857 51.506587982177734 114.08540344238281 60.29948425292969 121.21923828125 8 0.5070 62.850830078125 116.72383880615234 67.71094512939453 120.38460540771484 11 0.4931 21.695070266723

Evaluating:   0%|          | 2/1757 [00:21<4:40:04,  9.58s/it]

{'image_id': 'aa9c3097e6d2df7216be1a1642f683c9', 'PredictionString': '14 1.0 0 0 1 1'}
{'image_id': 'bb541cb8a793063968903bf3fc5b68f7', 'PredictionString': '0 0.9770 122.80204772949219 39.90216064453125 142.19229125976562 63.13593292236328 3 0.9733 104.5771484375 89.35327911376953 171.29122924804688 114.15386199951172 11 0.5260 131.34640502929688 23.44124412536621 158.0930633544922 31.16642951965332 11 0.4303 80.11498260498047 23.77183723449707 102.4603500366211 34.76554489135742 11 0.4264 148.18934631347656 23.838544845581055 159.42930603027344 33.12248611450195'}
{'image_id': '0101ad90f31ddb8fb24e9935a3dac9db', 'PredictionString': '14 1.0 0 0 1 1'}
{'image_id': '2a96f7ac3c6c3d856692929767006892', 'PredictionString': '14 1.0 0 0 1 1'}


Evaluating:   0%|          | 3/1757 [00:24<3:14:20,  6.65s/it]

...

{'image_id': '924eb4734344e5808df5d15855d68178', 'PredictionString': '3 0.8390 99.61537170410156 115.13721466064453 176.63262939453125 154.53826904296875 11 0.8135 129.89785766601562 33.765625 159.8004608154297 42.3607063293457 0 0.7587 119.29157257080078 67.6592025756836 142.740478515625 93.09407043457031 13 0.7283 136.2552947998047 38.08533477783203 152.74403381347656 60.71486282348633 13 0.5899 69.77583312988281 62.492610931396484 80.35899353027344 72.8195571899414 11 0.5405 146.41334533691406 33.60192108154297 160.74952697753906 42.022579193115234 13 0.5123 69.66510009765625 37.898101806640625 101.06761169433594 73.59498596191406 11 0.5063 130.63314819335938 34.201683044433594 151.2346954345703 39.56513214111328 6 0.4959 149.1693115234375 101.7500228881836 186.1744384765625 153.94065856933594 11 0.4736 135.49087524414062 33.27254104614258 166.3207244873047 48.4268913269043 11 0.4488 76.66967010498047 34.88661193847656 104.35174560546875 44.00009536743164 13 0.4312 139.4160003662109

Evaluating:   1%|          | 11/1757 [00:54<1:57:07,  4.03s/it]

{'image_id': 'aff9b5fa191d69543827088db17e6aa8', 'PredictionString': '8 0.6133 76.33366394042969 56.36619186401367 108.37760162353516 84.34020233154297 0 0.5801 80.88573455810547 55.35158920288086 119.66815948486328 86.25518798828125 11 0.5401 134.32411193847656 26.7628116607666 158.3924102783203 32.49943923950195 11 0.4683 126.85203552246094 26.126245498657227 161.6706085205078 34.7349739074707 8 0.4579 40.38801956176758 101.29743957519531 86.9546890258789 140.22438049316406'}
{'image_id': '8f0256af9634bee76dccf87a76b03c0f', 'PredictionString': '14 1.0 0 0 1 1'}
{'image_id': '84e9f232df2236e9dc7f7f472ff26bee', 'PredictionString': '8 0.5823 77.2928237915039 63.61411666870117 100.47843170166016 86.74855041503906 0 0.4170 77.31869506835938 62.28225326538086 104.62646484375 88.93020629882812'}
{'image_id': '5b96a79b2f850d88b7b265f777bc8da8', 'PredictionString': '14 1.0 0 0 1 1'}


Evaluating:   1%|          | 11/1757 [01:34<4:09:35,  8.58s/it]


KeyboardInterrupt: 

In [ ]:
""" model = ChestXrayLightningModel.load_from_checkpoint(
    "/Users/alexis/Cranfield/AI/assignment/Faster-R-CNN/model/epoch=9-step=9260.ckpt",
    num_classes=15,
)

dataModule.setup("test")    
train_acc = trainer.test(dataloaders=dataModule.test_dataloader(), model=model) """

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def show_image_with_boxes(image, boxes, labels=None, figsize=(10, 10)):
    """
    Visualize an image with its bounding boxes.

    Parameters:
    - image: a PIL.Image or numpy array of the image.
    - boxes: a list of bounding boxes, each defined by a list of four values [x_min, y_min, x_max, y_max].
    - labels: (Optional) a list of labels for each bounding box.
    - figsize: size of the figure to display.
    """
    # If the image is a torch tensor, convert it to a numpy array
    if torch.is_tensor(image):
        image = image.numpy().transpose((1, 2, 0))
    elif isinstance(image, Image.Image):
        image = np.array(image)

    # Normalize the image array to 0-1 range if it's not already
    if image.max() > 1.0:
        image = image / 255

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.imshow(image)

    for i, box in enumerate(boxes):
        x_min, y_min, x_max, y_max = box
        width, height = x_max - x_min, y_max - y_min

        rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2,
                                 edgecolor='r', facecolor='none')
        ax.add_patch(rect)

        if labels is not None and i < len(labels):
            label = labels[i]
            plt.text(x_min, y_min, str(label), color='white', fontsize=12, bbox=dict(facecolor='red', alpha=0.5))

    plt.axis('off')
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision

dataModule.setup("fit")

for images, targets in dataModule.train_dataloader():  
    break



In [ ]:
image = images[2]
target = targets[2]

boxes = target['boxes'].numpy()  # Convert tensor to numpy if it's not already
labels = target['labels'].numpy()  # Assuming you want to visualize labels as welltarget

In [ ]:
# Now, visualize the image with bounding boxes
show_image_with_boxes(image, boxes, labels)

In [ ]:
import pandas as pd
pred = pd.read_csv("/Users/alexis/Cranfield/AI/assignment/Faster-R-CNN/example_pred.csv")
pred.head()

In [ ]:
def parse_prediction_string(prediction_string):
    """Parse la chaîne de prédiction pour extraire les bounding boxes et les scores."""
    items = prediction_string.split()
    predictions = []

    for i in range(0, len(items), 6):
        class_id = int(items[i])
        score = float(items[i + 1])
        bbox = [float(coord) for coord in items[i + 2:i + 6]]
        predictions.append({'class_id': class_id, 'score': score, 'bbox': bbox})
    
    return predictions

In [ ]:
def filter_predictions_by_score(predictions, score_threshold=0.5):
    """Filtrer les prédictions pour ne conserver que celles avec un score supérieur au seuil."""
    return [pred for pred in predictions if pred['score'] > score_threshold]


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def show_image_with_predictions(image, predictions):
    """Afficher l'image avec les bounding boxes filtrées selon le score."""
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(image, cmap='gray')

    for pred in predictions:
        x_min, y_min, x_max, y_max = pred['bbox']
        width, height = x_max - x_min, y_max - y_min
        rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(x_min, y_min, f"{pred['class_id']}: {pred['score']:.2f}", color='white', fontsize=12, bbox=dict(facecolor='red', alpha=0.5))

    plt.axis('off')
    plt.show()


In [ ]:
import logging
import h5py

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

def get_pixel_array_from_hdf5(hdf5_path, filename):
    """
    Retrieves the pixel array for a given filename from an HDF5 file.

    Parameters:
    - hdf5_path: Path to the HDF5 file.
    - filename: The original file name of the DICOM image.

    Returns:
    - A numpy array of the pixel data if found; None otherwise.
    """
    with h5py.File(hdf5_path, "r") as hdf5_file:
        # Attempt to access the dataset directly by filename.
        # Adjust this part if a more complex naming convention is used.
        unique_filename = None
        for key in hdf5_file.keys():
            if filename in key:
                unique_filename = key
                break
                
        if unique_filename is not None:
            pixel_array = hdf5_file[unique_filename][:]
            return pixel_array
        else:
            logging.error(f"File {filename} not found in HDF5.")
            return None

In [ ]:
import os
import h5py

with h5py.File("/Volumes/ALEXIS/ai_project_cranfield/dicom_images_final.hdf5", "r") as hdf5_file:
    # Count the number of datasets (keys) directly
    num_datasets = len(hdf5_file.keys())

# Print the number of datasets
print(num_datasets)

In [ ]:
filename = "fad39dc356aaa2da58470c6daaba8112"  # This should be the base name of the file you're looking for.
pixel_array = get_pixel_array_from_hdf5("/Volumes/ALEXIS/ai_project_cranfield/dicom_images_final.hdf5", filename+".dicom")
plt.imshow(pixel_array, cmap="gray")

# print the size of the image
original_size = pixel_array.shape[:2]  # (hauteur, largeur)
print(original_size)

In [ ]:
pred = pred[pred['image_id'] == filename]

In [ ]:
pred 

In [ ]:
# Parsez les prédictions
predictions = parse_prediction_string(pred['PredictionString'].values[0])

# Filtrez selon un score de seuil, par exemple 0.9
filtered_predictions = filter_predictions_by_score(predictions, 0.9)

boxes = [pred['bbox'] for pred in filtered_predictions]

In [ ]:
def denormalize_bboxes(boxes, original_size, target_size):
    """
    Dénormalise les bounding boxes selon la taille cible de l'image.

    :param boxes: Liste des bounding boxes normalisées [x_min, y_min, x_max, y_max].
    :param original_size: Tuple de la taille originale de l'image (height, width).
    :param target_size: Tuple de la taille cible de l'image (height, width).
    :return: Liste des bounding boxes dénormalisées selon la taille cible.
    """
    denormalized_boxes = []
    for box in boxes:
        x_min, y_min, x_max, y_max = box
        x_min = x_min * target_size[1] / original_size[1]
        y_min = y_min * target_size[0] / original_size[0]
        x_max = x_max * target_size[1] / original_size[1]
        y_max = y_max * target_size[0] / original_size[0]
        denormalized_boxes.append([x_min, y_min, x_max, y_max])
    return denormalized_boxes

# Dénormalisez les bounding boxes
denormalized_boxes = denormalize_bboxes(boxes=boxes, original_size=original_size, target_size=(224, 224))

# Update the filtered predictions with the denormalized boxes
for i, pred in enumerate(filtered_predictions):
    pred['bbox'] = denormalized_boxes[i]

In [ ]:
show_image_with_predictions(pixel_array, filtered_predictions)